In [114]:
import pandas as pd

In [115]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/list-for-rent"

beds_min = 0
price_max = 5000
price_min = 800
# prop_type = "single_family"
baths_min = 0
radius = 1.5
city = "Denver"
state_code = "CO"
zip_code = 80206

querystring = {"beds_min":beds_min,
               "price_max":price_max,
#                "prop_type":prop_type,
               "sort":"relevance",
               "baths_min":baths_min,
               "radius":radius,
               "price_min":price_min,
               "city":city,
               "state_code":state_code,
               "limit":"200",
               "postal_code": zip_code,
               "offset":"0"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "bdf6bde97amsh2f0d8da5849d952p1e4163jsn48237c3a4f33"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()


In [116]:
rental_homes = data["properties"]

In [117]:
denver_hoods = pd.read_csv('denver.hoods.csv')
denver_hoods.set_index(["ID"], inplace = True)
denver_hoods

,City,Walk Score,Transit Score,Bike Score,Population
ID,,,,,
1,Downtown,93,89,94,15382
2,Capitol Hill,93,66,96,14685
3,Five Points,90,73,97,12709
4,Baker,86,60,87,4875
5,Cheesman Park,85,53,90,7984
...,...,...,...,...,...
70,Bear Valley,33,38,52,9926
71,Indian Creek,29,38,64,3261
72,Fort Logan,24,29,44,9211


In [118]:
# neighborhood = rental_homes[195]
# neighborhood

In [119]:
from math import sin, cos, sqrt, atan2, radians
counter = 0
address_list = []
beds_min_list = []
beds_max_list = []
baths_min_list = []
baths_max_list = []
lat_list = []
lon_list =[]
distance_list = []
price_min_list = []
price_max_list = []
sqft_min_list = []
sqft_max_list = []
neighborhood_list = []
# photos_array = {}

for listing in rental_homes:
#     try:
        try:
            address = rental_homes[counter]["address"]["line"]
            
        except:
            address = "N/A"
        address_list.append(address)
        try:
            beds_min = rental_homes[counter]["community"]["beds_min"]      
            beds_max = rental_homes[counter]["community"]["beds_max"]
        except:
            beds_min = rental_homes[counter]["beds"]
            beds_max = rental_homes[counter]["beds"]
        
        
        beds_min_list.append(beds_min)
        beds_max_list.append(beds_max)
        
        try:
            try:
                sqft_min = rental_homes[counter]["community"]["sqft_min"]      
                sqft_max = rental_homes[counter]["community"]["sqft_max"]
            except:
                sqft_min = rental_homes[counter]["sqft"]
                sqft_max = rental_homes[counter]["sqft"]

        except:
            sqft_min = "N/A"
            sqft_max = "N/A"
#             print(address)
#             print(counter)
        sqft_min_list.append(sqft_min)
        sqft_max_list.append(sqft_max)
        
        try:
            try:
                baths_min = rental_homes[counter]["community"]["baths_min"]      
                baths_max = rental_homes[counter]["community"]["baths_max"]
            except:
                baths_min = rental_homes[counter]["baths"]
                baths_max = rental_homes[counter]["baths"]
        except:
            baths_min = beds_min
            baths_max = beds_max
    #             print(counter)
            
        baths_min_list.append(baths_min)
        baths_max_list.append(baths_max)
        
        
        lat = rental_homes[counter]["address"]["lat"]
        lon = rental_homes[counter]["address"]["lon"]
        try:
            neighborhood = rental_homes[counter]["address"]["neighborhood_name"]
        except:
            neighborhood = "Unknown"
        neighborhood_list.append(neighborhood)
        
        try:
            price_min = rental_homes[counter]["price"]
            price_max = rental_homes[counter]["price"]
        except(KeyError):
            price_min = rental_homes[counter]["community"]["price_min"]      
            price_max = rental_homes[counter]["community"]["price_max"]
        lat_list.append(lat)
        lon_list.append(lon)
        
        price_min_list.append(price_min)
        price_max_list.append(price_max)
#         photos_list = rental_homes[counter]["photos"][0]
        R = 6373.0
        lat1 = radians(39.728068)
        lon1 = radians(-104.959956)
        lat2 = radians(lat)
        lon2 = radians(lon)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        distance_list.append(distance)
        counter = counter+1
#     except (KeyError, IndexError):
#         price_min = rental_homes[counter]["price"]
#         price_max = rental_homes[counter]["price"]

    


In [120]:
df = pd.DataFrame()

In [121]:
df["Address"] = address_list
df["Beds_Min"] = beds_min_list
df["Beds_Max"] = beds_max_list
df["Baths_Min"] = baths_min_list
df["Baths_Max"] = baths_max_list
df["Sqft_Min"] = sqft_min_list
df["Sqft_Max"] = sqft_max_list
df["Lat"] = lat_list
df["Lon"] = lon_list
df["Distance"] = distance_list
df["Rent"] = price_min_list
df["Rent_max"] = price_max_list
df["Neighborhood"] = neighborhood_list
df = df.dropna()
# df["Photos"] = photos_list

In [122]:
df = df.sort_values(["Distance"])

In [123]:
df_merge = df.merge(denver_hoods,left_on = "Neighborhood", right_on = "City",how = "left")

In [124]:
df_merge

,Address,Beds_Min,Beds_Max,Baths_Min,Baths_Max,Sqft_Min,Sqft_Max,Lat,Lon,Distance,Rent,Rent_max,Neighborhood,City,Walk Score,Transit Score,Bike Score,Population
0,2190 E 11th Ave,0,3,1.0,2.0,516,1331,39.733230,-104.961560,0.590335,1638,3341,Cheesman Park,Cheesman Park,85.0,53.0,90.0,7984.0
1,2185 E 11th Ave # 5-512,0,0,1.0,1.0,N/A,N/A,39.733579,-104.961253,0.622947,1638,1638,Cheesman Park,Cheesman Park,85.0,53.0,90.0,7984.0
2,2185 E 11th Ave # 4-407,1,1,1.0,1.0,N/A,N/A,39.733579,-104.961253,0.622947,1807,1807,Cheesman Park,Cheesman Park,85.0,53.0,90.0,7984.0
3,1137 N Columbine St Unit 205,1,1,1.0,1.0,N/A,N/A,39.734497,-104.957776,0.739012,1085,1085,Congress Park,Congress Park,80.0,51.0,88.0,10239.0
4,2315 E 12th Ave,1,2,1.0,1.0,599,828,39.735298,-104.959447,0.805370,1300,1600,Congress Park,Congress Park,80.0,51.0,88.0,10239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,980 S Dexter St Apt 17,0,0,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,995,995,Glendale,NaN,NaN,NaN,NaN,NaN
195,980 S Dexter St Apt 10,1,1,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,1115,1115,Glendale,NaN,NaN,NaN,NaN,NaN
196,980 S Dexter St Apt 24,0,0,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,995,995,Glendale,NaN,NaN,NaN,NaN,NaN
197,980 S Dexter St Apt 15,1,1,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,1095,1095,Glendale,NaN,NaN,NaN,NaN,NaN


In [125]:
df_merge.to_csv('excelview.csv')


In [126]:
df

,Address,Beds_Min,Beds_Max,Baths_Min,Baths_Max,Sqft_Min,Sqft_Max,Lat,Lon,Distance,Rent,Rent_max,Neighborhood
7,2190 E 11th Ave,0,3,1.0,2.0,516,1331,39.733230,-104.961560,0.590335,1638,3341,Cheesman Park
196,2185 E 11th Ave # 5-512,0,0,1.0,1.0,N/A,N/A,39.733579,-104.961253,0.622947,1638,1638,Cheesman Park
194,2185 E 11th Ave # 4-407,1,1,1.0,1.0,N/A,N/A,39.733579,-104.961253,0.622947,1807,1807,Cheesman Park
190,1137 N Columbine St Unit 205,1,1,1.0,1.0,N/A,N/A,39.734497,-104.957776,0.739012,1085,1085,Congress Park
59,2315 E 12th Ave,1,2,1.0,1.0,599,828,39.735298,-104.959447,0.805370,1300,1600,Congress Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,980 S Dexter St Apt 17,0,0,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,995,995,Glendale
140,980 S Dexter St Apt 10,1,1,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,1115,1115,Glendale
130,980 S Dexter St Apt 24,0,0,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,995,995,Glendale
128,980 S Dexter St Apt 15,1,1,1.0,1.0,N/A,N/A,39.698658,-104.932014,4.051816,1095,1095,Glendale


In [127]:
from math import sin, cos, sqrt, atan2, radians


    

# approximate radius of earth in km
R = 6373.0

lat1 = radians(39.663750)
lon1 = radians(-105.053680)
lat2 = radians(52.406374)
lon2 = radians(16.9251681)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
print("Should be:", 278.546, "km")

Result: 8354.032533618552
Should be: 278.546 km
